In [1]:
import pandas as pd
from shapely.geometry import Point
import warnings
from shapely.wkt import loads
import trackintel as ti
from tqdm import tqdm

/usr/local/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.9/site-packages/trackintel/io/file.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/

In [2]:
stop_point_df = pd.read_csv('location.csv')

In [3]:
stop_point_df

,id,user_id,center,extent
0,0,0,POINT (-81.7528984799999989 30.2712044549999995),POLYGON ((-81.7538813735643686 30.271544781744...
1,1,0,POINT (-81.8217049999999944 30.2850999999999999),POLYGON ((-81.8217000000000070 30.286140281844...
2,2,0,POINT (-81.6755752600000022 30.3649654300000016),POLYGON ((-81.6745341296104499 30.364965430000...
3,3,0,POINT (-81.6028094850000087 30.1917907599999999),POLYGON ((-81.6028189700000013 30.191781519999...
4,4,0,POINT (-81.6623927299999934 30.4500015750000017),POLYGON ((-81.6624000000000052 30.449999999999...
...,...,...,...,...
36838,36838,5167,POINT (-81.6892980000000080 30.3958003333333338),POLYGON ((-81.6893010000000004 30.395800000000...
36839,36839,5168,POINT (-81.7528999999999968 30.2712000000000003),POLYGON ((-81.7518598654911557 30.271200000000...
36840,36840,5170,POINT (-81.5872957452777854 30.3667969686111121),POLYGON ((-81.5872935000000012 30.366795000000...
36841,36841,5170,POINT (-81.5921142857142740 30.3675916785714257),POLYGON ((-81.5920149999999893 30.367564999999...


In [4]:
retail_pd = pd.read_csv('retails_Jacksonville_FL.csv')
retail_pd

,Unnamed: 0,ObjectId,Store_Name,Address,Address_Li,City,State,Zip5,Zip4,County,Longitude,Latitude,geometry
0,1105,1106,The Pit Stop 2,996 Saint Johns Bluff Rd N,NaN,Jacksonville,FL,32225,8386.0,DUVAL,-81.523949,30.338488,POINT (-81.52394899999996 30.33848800000004)
1,1214,1215,James Food Mart,2651 Jammes Rd,NaN,Jacksonville,FL,32210,3751.0,DUVAL,-81.747391,30.271931,POINT (-81.74739099999994 30.271931000000052)
2,1856,1857,Vibe Smoke & Mini Mart,619 University Blvd N,NaN,Jacksonville,FL,32211,7037.0,DUVAL,-81.604263,30.326391,POINT (-81.60426299999995 30.326391000000058)
3,1882,1883,IFG Jacksonville 344,3100 Hilton St,NaN,Jacksonville,FL,32209,2750.0,DUVAL,-81.715515,30.365610,POINT (-81.71551499999998 30.36561000000006)
4,2323,2324,Jacksonville 76 Petroleum Llc,5971 Ramona Blvd,NaN,Jacksonville,FL,32205,4755.0,DUVAL,-81.751587,30.314375,POINT (-81.75158699999997 30.31437500000004)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
844,244425,244426,J & W Discount,4045 Post St,NaN,Jacksonville,FL,32205,5388.0,DUVAL,-81.719048,30.311987,POINT (-81.71904799999999 30.311987000000045)
845,244595,244596,Ajs Seafood Llc 2 2,725 Edgewood Ave N,NaN,Jacksonville,FL,32254,3013.0,DUVAL,-81.730835,30.331091,POINT (-81.73083499999996 30.33109100000007)
846,244629,244630,Abbey Food Store,6164 Cleveland Rd,NaN,Jacksonville,FL,32209,1923.0,DUVAL,-81.715858,30.377815,POINT (-81.71585799999997 30.377815000000055)
847,244891,244892,Soriya Food Market,3561 Emerson St,NaN,Jacksonville,FL,32207,5628.0,DUVAL,-81.628120,30.291224,POINT (-81.62811999999997 30.291224000000057)


In [5]:
def is_close(dist, latA1, lonA2, latB1, lonB2):
    # Create shapely Point objects from the locations
    point1 = Point(lonA2, latA1)
    point2 = Point(lonB2, latB1)
    
    # Calculate distance between points in meters
    distance = point1.distance(point2) * 111319.9
    
    # Return True if distance is less than or equal to 5 meters
    return distance <= dist

In [6]:
count = 0
warnings.simplefilter(action='ignore', category=Warning)
data_to_append_list = []
for sp in tqdm(stop_point_df.itertuples(), total=len(stop_point_df)):
    p = loads(sp[3])
    lat = p.y
    lon = p.x
    
    for row in retail_pd.itertuples():
        if is_close(100, float(row[12]), float(row[11]), lat, lon):
            count += 1
            
            data_to_append = {'id': sp[1], 'user_id': sp[2], 'lat': lat, 'lon': lon, 'retail_id': row[1], 
                                'retail_lat': float(row[12]), 'retail_lon': float(row[11])}
            data_to_append_list.append(data_to_append)
            
            break

new_df = pd.DataFrame.from_records(data_to_append_list)
print('Point close to retail: ', count)

100%|██████████| 36843/36843 [13:14<00:00, 46.38it/s]

Point close to retail:  2126


In [7]:
new_df

,id,user_id,lat,lon,retail_id,retail_lat,retail_lon
0,48,3,30.320221,-81.458439,14500,30.320408,-81.458824
1,54,3,30.319797,-81.461897,63990,30.319975,-81.461052
2,59,3,30.360386,-81.502458,117988,30.360104,-81.501938
3,82,3,30.287588,-81.464670,153229,30.287714,-81.464462
4,90,7,30.198132,-81.823522,57475,30.197430,-81.823288
...,...,...,...,...,...,...,...
2121,36707,5109,30.332180,-81.655647,69770,30.332350,-81.656479
2122,36723,5118,30.290736,-81.592322,88755,30.290123,-81.591957
2123,36744,5126,30.319097,-81.472843,51004,30.319002,-81.472450
2124,36779,5140,30.204381,-81.739914,185564,30.204830,-81.739655


In [8]:
new_df.to_csv('food_inference.csv')